In [1]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

In [2]:
class Memory:
    def __init__(self) -> None:
        self.memory = []
    
    def add(self, new_memory):
        self.memory.append(new_memory)

        if len(self.memory) > 2:
            self.memory.pop(0)

In [3]:
class DataAgent:
    def __init__(self, df) -> None:

        model_name = 'llama3.2'
        self.df = df

        self.memory = Memory()

        self.df_llm = OllamaLLM(model=model_name, temperature=0)
        self.df_template = """
        Historial de conversación: {memory}
        Estructura del DataFrame: {df_structure}
        Pregunta: {question}
        Nombre de la variable del DataFrame: df
        Respuesta: Por favor, proporciona solamente el código de Python necesario para resolver la pregunta basada en la estructura del DataFrame de Pandas proporcionada, 
        debes guardar la respuesta en una variable llamada resultado, sin imprimir nada.
        En caso de requerir multiples respuestas, incluyelas en un diccionario llamado resultado.
        No me expliques nada, no incluyas '''pyton''' ni nada de eso. Solamente codigo ejecutable. 
        Nunca, bajo ninguna circunstancia me des algo así: ```python [codigo] ```, solamente el codigo sin nada más.
        """
        self.df_prompt = ChatPromptTemplate.from_template(self.df_template)
        self.df_chain = self.df_prompt | self.df_llm

        self.humanize_llm = OllamaLLM(model=model_name, temperature=0)
        self.humanize_template = """
        Historial de conversación: {memory}
        Pregunta: {question}
        Resultado: {result}
        Estructura del DataFrame: {df_structure}
        Contexto: Previamente, se calculó el resultado a una pregunta sobre un dataframe con la estructura proporcionada. 
        Eres un asistente digital amigable llamado Bimbot, tu trabajo es darle un formato humano al resultado.
        Responde solamente basandote en el resultado obtenido.
        Respuesta: Por favor, escribe como un humano lo haría para responder la pregunta hecha previamente, basandote en el resultado obtenido.
        """
        self.humanize_prompt = ChatPromptTemplate.from_template(self.humanize_template)
        self.humanize_chain = self.humanize_prompt | self.humanize_llm

    def add_to_memory(self, question, result):
        self.memory.add(f'|Pregunta: {question}, Respuesta: {result}|')

    def get_code(self, question):
        res = self.df_chain.invoke({            
            'df_structure': self.df.dtypes,
            'question': question,
            'memory': self.memory.memory
        })
        return res
    
    def get_result(self, code):
        context = {'df': self.df.copy()}
        try:
            exec(code, context)
            return context['resultado']
        except Exception as e:
            return f'Ocurrió un error al ejecutar el codigo: {e} | Intentar otra instrucción'
    
    def humanize_result(self, question, result):
        res = self.humanize_chain.invoke({
            'question': question,
            'result': result,
            'df_structure': self.df.dtypes,
            'memory': self.memory.memory
        })
        return res
    
    def ask_question(self, question):
        code = self.get_code(question)
        print(code)
        result = self.get_result(code)
        print(result)
        print(self.memory.memory)
        human_result = self.humanize_result(question, result)
        self.add_to_memory(question, result)
        return human_result

In [4]:
import pandas as pd
import numpy as np

path = '../../../Utils/CC.xlsx'
x = pd.read_excel(path)
x = x[x['Descripcion de articulo'].notnull()]
articulos = x['Descripcion de articulo'].unique()

np.random.seed(42)
num_rows = 250
data = {
    'Rack': [chr(np.random.randint(65, 91)) for _ in range(num_rows)],
    'Columna': np.random.randint(1, 53, size=num_rows),
    'Fila': np.random.randint(1, 4, size=num_rows),
    'Cantidad': np.random.randint(1, 49, size=num_rows),
    'Descripcion del articulo': np.random.choice(articulos, size=num_rows)
}
df = pd.DataFrame(data)
df

def rack_to_number(rack_str):
    try:
        n = ord(rack_str.upper()) - ord('A') + 1
        return n
    except (AttributeError, TypeError):
      return

df = df.drop_duplicates(subset=['Rack', 'Columna'])
df['n'] = df['Rack'].apply(rack_to_number)
df = df[df['n'] != 0]
df['Fila'] = df['Fila'].astype(int)
df['n'] = df['n'].astype(int)
df['Ubicacion'] = df['Rack'] + "-" + df['n'].astype(str) + "-" + df['Columna'].astype(str) + "-" + df['Fila'].astype(str)

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
da = DataAgent(df)

@app.route('/ask', methods=['POST'])
def ask_question():
    data = request.get_json()
    print(data)
    question = data.get('question')
    
    answer = da.ask_question(question)
    
    return jsonify({'answer': answer})

if __name__ == '__main__':
    app.run(debug=False, port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [10/Nov/2024 00:44:18] "OPTIONS /ask HTTP/1.1" 200 -


{'question': 'Cual rack tiene mayor cantidad?'}
resultado = df.loc[df['Cantidad'].idxmax()]['Rack']
C
[]


127.0.0.1 - - [10/Nov/2024 00:44:23] "POST /ask HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2024 00:44:29] "OPTIONS /ask HTTP/1.1" 200 -


{'question': 'Cuanta cantidad tiene ese rack?'}
resultado = df['Cantidad'].max()
48
['|Pregunta: Cual rack tiene mayor cantidad?, Respuesta: C|']


127.0.0.1 - - [10/Nov/2024 00:44:34] "POST /ask HTTP/1.1" 200 -
127.0.0.1 - - [10/Nov/2024 00:45:06] "OPTIONS /ask HTTP/1.1" 200 -


{'question': 'Cuanta cantidad tiene ese Rack en total?'}
resultado = df['Cantidad'].sum()
5636
['|Pregunta: Cual rack tiene mayor cantidad?, Respuesta: C|', '|Pregunta: Cuanta cantidad tiene ese rack?, Respuesta: 48|']


127.0.0.1 - - [10/Nov/2024 00:45:10] "POST /ask HTTP/1.1" 200 -
